# RAWG — Business-focused EDA
This notebook performs a business- and research-centric exploratory data analysis of the RAWG cleaned dataset (`data/processed/rawg_cleaned.csv`).
Contract:
- Input: `data/processed/rawg_cleaned.csv` (filtered to 2024-11-11 -> 2025-11-11)
- Outputs: interactive Plotly charts for executive dashboards; engineered features saved when useful.

In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
pd.set_option('display.max_columns', 100)

In [ ]:
PATH = '/home/jubaer/Downloads/GOTY 2025 analysis/data/processed/rawg_cleaned.csv'
assert os.path.exists(PATH), f'Missing {PATH} - run cleaners first'
df = pd.read_csv(PATH, dtype=str)
# normalize release_date
df['release_date'] = pd.to_datetime(df.get('release_date', ''), errors='coerce')
df['ratings'] = pd.to_numeric(df.get('ratings', None), errors='coerce')
df['metacritic'] = pd.to_numeric(df.get('metacritic', None), errors='coerce')
# shorten description for quick display
if 'description' in df.columns:
    df['description_short'] = df['description'].fillna('').str.slice(0, 200).str.replace('\\n', ' ')
df.shape


In [ ]:
# Quick overview
display(df.head(5))
display(df.info())

## Feature engineering: explode list fields for analysis (genres, tags, platforms)
We'll create exploded tables for genre- and platform-level aggregates which are useful in dashboards (top genres, platform share, tag insights).

In [ ]:
for col in ['genres', 'tags', 'platforms']:
    if col not in df.columns:
        df[col] = ''
    df[col] = df[col].fillna('').astype(str)
    df[col + '_list'] = df[col].apply(lambda s: [x.strip() for x in s.split('|') if x.strip()])

# exploded genres for counting
df_genre = df.explode('genres_list')
df_genre['genres_list'] = df_genre['genres_list'].fillna('')
genre_counts = df_genre[df_genre['genres_list'] != '']['genres_list'].value_counts().reset_index()
genre_counts.columns = ['genre','count']
genre_counts.head()

In [ ]:
# Top genres (business view)
fig = px.bar(genre_counts.head(20), x='genre', y='count', title='Top genres (by number of releases)',
             text='count', template='plotly_white')
fig.update_layout(xaxis_tickangle=-45, height=480)
fig.show()

### Ratings distribution and summary statistics
Understand user rating distribution (RAWG ratings) and Metacritic where available.

In [ ]:
# Ratings histogram
fig = px.histogram(df, x='ratings', nbins=40, title='Distribution of RAWG user ratings', template='plotly_white')
fig.update_layout(height=420)
fig.show()

# Metacritic where present
if df['metacritic'].notna().any():
    fig2 = px.histogram(df, x='metacritic', nbins=40, title='Metacritic score distribution (where available)', template='plotly_white')
    fig2.update_layout(height=420)
    fig2.show()

### Platform share and release timeline
A business view of platform distribution and releases over the year.

In [ ]:
# Platform share (explode platforms)
df_plat = df.explode('platforms_list')
df_plat['platforms_list'] = df_plat['platforms_list'].fillna('')
plat_counts = df_plat[df_plat['platforms_list'] != '']['platforms_list'].value_counts().reset_index()
plat_counts.columns = ['platform','count']
fig = px.pie(plat_counts.head(15), names='platform', values='count', title='Platform share (top platforms)', template='plotly_white')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

# Releases over time (by month)
df['release_month'] = df['release_date'].dt.to_period('M').dt.to_timestamp()
timeline = df.groupby('release_month').size().reset_index(name='count')
fig = px.line(timeline, x='release_month', y='count', title='Releases by month', markers=True, template='plotly_white')
fig.update_layout(xaxis_title='Month', height=420)
fig.show()

### Top titles and short profiles
Create a compact executive table showing top-rated and most-discussed titles with key attributes.

In [ ]:
top_rated = df.sort_values(by='ratings', ascending=False).head(20)[['name','release_date','ratings','metacritic','platforms','genres']].copy()
top_rated['ratings'] = top_rated['ratings'].round(2)
top_rated.reset_index(drop=True, inplace=True)
top_rated

## Save engineered features (optional)
Saving exploded genre and platform tallies to `data/processed/` so they can be reused by dashboards or Tableau.

In [ ]:
os.makedirs('data/processed', exist_ok=True)
genre_counts.to_csv('data/processed/rawg_genre_counts.csv', index=False)
plat_counts.to_csv('data/processed/rawg_platform_counts.csv', index=False)
print('Wrote rawg_genre_counts and rawg_platform_counts to data/processed')